In [4]:
# 🗝️ Mediastack key
MEDIASTACK_KEY = "6a524b08bdd857104ac3068d9b2a5c2d"

from google.colab import drive
import os, pathlib, shutil, glob, time

# 1. подключаем Drive
drive.mount("/content/drive", force_remount=False)
DRIVE = pathlib.Path("/content/drive/MyDrive/test")
TMP   = pathlib.Path("/content/RSS_tmp")

# 2. клон репозитория (RAM)
os.chdir("/content")
if TMP.exists(): shutil.rmtree(TMP)
!git clone -q --depth 1 https://github.com/vkalinovski/RSS_Project.git {TMP}

CODE = next((p.parent for p in TMP.rglob("mediastack_fetcher.py")), TMP)
print("📂 scripts:", CODE)

# 3. минимальные зависимости
!pip install -q feedparser requests python-dotenv pandas==2.2.2 matplotlib==3.8.4 nltk tqdm

# 4. переменные окружения
DRIVE.mkdir(parents=True, exist_ok=True)
os.environ["DB_PATH"] = str(DRIVE/"news.db")
(CODE/".env").write_text(f"MEDIASTACK_KEY={MEDIASTACK_KEY}\n")

# 5. запуск ETL
%cd {CODE}
!python mediastack_fetcher.py          || echo "⚠️ Mediastack step failed"
!python sentiment_analysis.py          || echo "⚠️ Sentiment step failed"
!python analyze.py                     || echo "⚠️ Analyze step failed"

# 6. перенос артефактов в Drive
for f in glob.glob("news.csv"):
    shutil.move(f, DRIVE/"news.csv")
if (CODE/"graphs").is_dir():
    if (DRIVE/"graphs").exists(): shutil.rmtree(DRIVE/"graphs")
    shutil.move(str(CODE/"graphs"), DRIVE/"graphs")

print("\n✅  Готово!  db/ и graphs/ →", DRIVE)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📂 scripts: /content/RSS_tmp/files
/content/RSS_tmp/files
Trump 2024-09: 0
Trump 2024-10: 0
Trump 2024-11: 0
Trump 2024-12: 0
Trump 2025-01: 100
Trump 2025-02: 100
Trump 2025-03: 100
Trump 2025-04: 100
Putin 2024-09: 0
Putin 2024-10: 0
Putin 2024-11: 0
Putin 2024-12: 0
Putin 2025-01: 100
Putin 2025-02: 100
Putin 2025-03: 100
Putin 2025-04: 100
Xi 2024-09: 0
Xi 2024-10: 0
Xi 2024-11: 0
Xi 2024-12: 0
Xi 2025-01: 100
Xi 2025-02: 89
Xi 2025-03: 86
Xi 2025-04: 100
✓ 76245 новых статей сохранено
✓ 14428 новых статей сохранено
✓ 23005 новых статей сохранено
✓ 75608 новых статей сохранено
sentiment: 100% 1144/1144 [00:00<00:00, 2084.39it/s]
✓ sentiment обновлён: 1144
/content/RSS_tmp/files/analyze.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 